In [5]:
import os
from pathlib import Path
from typing import List, Tuple, Callable

import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader


In [6]:
real_root= r"C:\Users\IT\Desktop\Video test seperated the audio\Real videos_only"
fake_root= r"C:\Users\IT\Desktop\Video test seperated the audio\videos_only fake"

def load_dataset(real_root: str, fake_root: str) -> List[Tuple[str, int]]:

    """
    real_root: folder with real videos
    fake_root: folder with fake videos
    returns: list of (video_path, label)
    """
    paths_and_labels: List[Tuple[str, int]] = []

    for root, label in [(real_root, 1), (fake_root, 0)]:
        root_path = Path(root)
        for p in root_path.rglob("*.mp4"):   # search all mp4s under each root
            paths_and_labels.append((str(p), label))

    return paths_and_labels


In [17]:
from typing import Any
import cv2 
dataset=load_dataset(real_root,fake_root)
def Video_analysis(dataset: list[tuple[str, int]]) -> list[dict[str, Any]]:
    dataset_loaded: list[dict[str, Any]] = []
    for video_path, label in dataset:
        try:
            cap = cv2.VideoCapture(video_path)

            # If video cannot be opened
            if not cap.isOpened():
                raise Exception("Video cannot be opened")

            # --- Extract metadata ---
            fps = cap.get(cv2.CAP_PROP_FPS)
            frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
            duration = frames / fps if fps > 0 else 0
            width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
            height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

            cap.release()

            # Save info to dataset
            dataset_loaded.append({
                "path": video_path,
                "label": label,
                "fps": fps,
                "frames": frames,
                "duration": duration,
                "width": width,
                "height": height
            })

        except Exception as e:
            print(f"Skipping {video_path} because: {e}")
            continue
    return dataset_loaded

video_data = Video_analysis(dataset)
print(video_data)

[{'path': 'C:\\Users\\IT\\Desktop\\Video test seperated the audio\\Real videos_only\\African\\men\\id00076\\00109.mp4', 'label': 1, 'fps': 25.0, 'frames': 251.0, 'duration': 10.04, 'width': 224.0, 'height': 224.0}, {'path': 'C:\\Users\\IT\\Desktop\\Video test seperated the audio\\Real videos_only\\African\\men\\id00166\\00010.mp4', 'label': 1, 'fps': 25.0, 'frames': 160.0, 'duration': 6.4, 'width': 224.0, 'height': 224.0}, {'path': 'C:\\Users\\IT\\Desktop\\Video test seperated the audio\\Real videos_only\\African\\men\\id00173\\00118.mp4', 'label': 1, 'fps': 25.0, 'frames': 149.0, 'duration': 5.96, 'width': 224.0, 'height': 224.0}, {'path': 'C:\\Users\\IT\\Desktop\\Video test seperated the audio\\Real videos_only\\African\\men\\id00366\\00118.mp4', 'label': 1, 'fps': 25.0, 'frames': 224.0, 'duration': 8.96, 'width': 224.0, 'height': 224.0}, {'path': 'C:\\Users\\IT\\Desktop\\Video test seperated the audio\\Real videos_only\\African\\men\\id00391\\00052.mp4', 'label': 1, 'fps': 25.0, 'fr

In [16]:
def compute_averages(video_data: list[dict[str, Any]]):
    if len(video_data) == 0:
        print("No valid videos to compute averages.")
        return

    avg_fps = sum(v["fps"] for v in video_data) / len(video_data)
    avg_frames = sum(v["frames"] for v in video_data) / len(video_data)
    avg_duration = sum(v["duration"] for v in video_data) / len(video_data)
    avg_width = sum(v["width"] for v in video_data) / len(video_data)
    avg_height = sum(v["height"] for v in video_data) / len(video_data)

    print("\n===== Average Video Statistics =====")
    print(f"Average FPS:       {avg_fps:.2f}")
    print(f"Average Frames:    {avg_frames:.2f}")
    print(f"Average Duration:  {avg_duration:.2f} seconds")
    print(f"Average Width:     {avg_width:.2f}px")
    print(f"Average Height:    {avg_height:.2f}px")
    print("===================================\n")


# ------------------------------
# Run analysis
# ------------------------------
dataset = load_dataset(real_root, fake_root)
video_data = Video_analysis(dataset)
compute_averages(video_data)


===== Average Video Statistics =====
Average FPS:       25.02
Average Frames:    147.78
Average Duration:  5.91 seconds
Average Width:     226.27px
Average Height:    226.22px



In [7]:
def sample_frames(video_path: str, num_frames: int) -> List[np.ndarray]:
    """Uniformly sample num_frames from the video."""
    cap = cv2.VideoCapture(video_path)
    frames: List[np.ndarray] = []

    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total <= 0:
        cap.release()
        return frames

    indices = np.linspace(0, total - 1, num_frames, dtype=int)
    idx_set = set(indices.tolist())

    idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if idx in idx_set:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
        idx += 1

    cap.release()
    return frames


def detect_face(frames: List[np.ndarray]) -> List[np.ndarray]:
    """Dummy face detector – currently just returns the original frames."""
    return frames


def preprocess_face(frames: List[np.ndarray]) -> List[np.ndarray]:
    """Resize to 224x224 and normalize to [0,1]."""
    processed: List[np.ndarray] = []
    for f in frames:
        f = cv2.resize(f, (224, 224))
        f = f.astype(np.float32) / 255.0
        processed.append(f)
    return processed


In [8]:
def preprocess_video_for_deepfake(
    video_path: str,
    sample_frames_func: Callable[[str, int], List[np.ndarray]],
    detect_face_func: Callable[[List[np.ndarray]], List[np.ndarray]],
    preprocess_face_func: Callable[[List[np.ndarray]], List[np.ndarray]],
    num_frames: int,
) -> torch.Tensor:
    # 1) sample frames
    frames = sample_frames_func(video_path, num_frames)

    if len(frames) == 0:
        # fallback: all zeros if video cannot be read
        return torch.zeros(num_frames, 3, 224, 224)

    # 2) detect faces and preprocess
    frames = detect_face_func(frames)
    frames = preprocess_face_func(frames)

    # 3) enforce fixed length
    if len(frames) > num_frames:
        frames = frames[:num_frames]
    elif len(frames) < num_frames:
        last = frames[-1]
        frames = frames + [last] * (num_frames - len(frames))

    # 4) to tensor (T, C, H, W)
    arr = np.stack(frames)              # (T, H, W, C)
    tensor = torch.from_numpy(arr).permute(0, 3, 1, 2).float()
    return tensor


In [11]:
from torch.utils.data import Dataset

class VideoDataset(Dataset):
    def __init__(self, paths_and_labels, num_frames,
                 sample_frames_func, detect_face_func, preprocess_face_func):
        self.data = paths_and_labels
        self.num_frames = num_frames
        self.sample_frames_func = sample_frames
        self.detect_face_func = detect_face
        self.preprocess_face_func = preprocess_face

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        video_path, label = self.data[idx]
        video_tensor = preprocess_video_for_deepfake(
            video_path,
            self.sample_frames_func,
            self.detect_face_func,
            self.preprocess_face_func,
            num_frames=self.num_frames,
        )
        return video_tensor, label


In [12]:
# 1) build (path, label) list
paths_and_labels = load_dataset(real_root, fake_root)
print("Total videos:", len(paths_and_labels))

# 2) create Dataset
num_frames = 32
video_dataset = VideoDataset(
    paths_and_labels=paths_and_labels,
    num_frames=num_frames,
    sample_frames_func=sample_frames,
    detect_face_func=detect_face,
    preprocess_face_func=preprocess_face,
)

# 3) create DataLoader (THIS is what you iterate over)
video_loader = DataLoader(video_dataset, batch_size=4, shuffle=True, num_workers=0)

# 4) check one batch
for videos, labels in video_loader:
    print("Batch videos shape:", videos.shape)  # expected: (B, T, C, H, W)
    print("Batch labels:", labels)
    break

Total videos: 10209
Batch videos shape: torch.Size([4, 32, 3, 224, 224])
Batch labels: tensor([0, 0, 0, 0])


In [15]:
# number of videos in the dataset
num_videos = len(video_dataset)
print("Num videos:", num_videos)

# shape of one video tensor
sample_video, sample_label = video_dataset[0]
print("One video shape:", sample_video.shape)


Num videos: 10209
One video shape: torch.Size([32, 3, 224, 224])
